# Modelado de tópicos con LDA

Los módulos a emplear en esta práctica son:

In [1]:
from nltk.corpus import PlaintextCorpusReader
import textacy, spacy, nltk
from stop_words import get_stop_words
import gensim
from gensim import corpora, models
import pyLDAvis.gensim

Se procede a la carga de documentos:

In [2]:
docs_dir = "./data/"
documentos = PlaintextCorpusReader(docs_dir, '.*') 

Los documentos presentes en el directorio son:

In [3]:
print(documentos.fileids()) 

['Abanico Lady Windermere.txt', 'Cultura.txt', 'Dorian Gray.txt', 'ONU.txt', 'brownie.txt', 'comida.txt', 'deporte.txt', 'queen.txt', 'rosas.txt', 'tenis.txt']


Cargamos el modelo de Spacy:

In [4]:
nlp = spacy.load('es_core_news_sm')

Adaptamos el formatos de los documentos para trabajar con Spacy y almacenamos en `idiomas` los idiomas de los textos:

In [5]:
procesados = []
idiomas = []
doc_list = []
for titulo in documentos.fileids():
    idiomas.append(textacy.text_utils.detect_language(documentos.raw(fileids=titulo)))
    doc_list.append(documentos.raw(fileids=titulo))
    procesados.append(nlp(documentos.raw(fileids=titulo)))

Se observa que todos los textos están en español:

In [6]:
idiomas

['es', 'es', 'es', 'es', 'es', 'es', 'es', 'es', 'es', 'es']

Se visualiza para comprobar la carga correcta uno de los textos:

In [7]:
procesados[3]

La Organización de las Naciones Unidas (ONU), o simplemente las Naciones Unidas (NN. UU.), es la mayor organización internacional existente. Se creó para mantener la paz y seguridad internacionales, fomentar relaciones de amistad entre las naciones, lograr la cooperación internacional para solucionar problemas globales y servir de centro que armonice las acciones de las naciones. Su sede está en Nueva York (Estados Unidos) y está sujeta a un régimen de extraterritorialidad. También tiene oficinas en Ginebra (Suiza), Nairobi (Kenia) y Viena (Austria).

La ONU se rige por la Carta de las Naciones Unidas, que entró en vigor el 24 de octubre de 1945 y se firmó el 25 de junio del mismo año en la ciudad estadounidense de San Francisco, por 51 países, pocos meses antes del final de la Segunda Guerra Mundial. En el preámbulo de la Carta se mencionan explícitamente las dos guerras mundiales.

Se observa la división de los textos en frases, palabras, se realiza el análisis POS, se extraen las formas normales y las entidades:

In [8]:
for procesado in procesados:
    
    print("\n Frases:")
    for sent in procesado.sents:
        print(sent.text)
        
    print("\n Tokens:")
    for token in procesado:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)
        
    print("\n Entidades:")
    for ent in procesado.ents:
        print(ent.text, ent.label_)
        
    


 Frases:
El abanico de Lady Windermere es una obra de teatro que se divide en cuatro actos de Oscar Wilde, estrenada el 20 de febrero de 1892 en el Teatro St James.


La obra comienza cuando Lady Windermere descubre que su marido puede estar manteniendo una relación con otra mujer.
La duquesa de Berwick es la que ha delatado al marido.
Lady Windermere se enfrenta a su esposo y le pide explicaciones, pero éste rechaza las acusaciones e invita a la supuesta amante, Mrs Erlynne, al baile de cumpleaños de su mujer.


Ultrajada por la infidelidad de su marido, Lady Windermere decide abandonarlo para seguir a Lord Darlington, un amigo que acaba de confesarle su amor.


Tras descubrir lo sucedido, Mrs Erlynne sigue a Lady Windermere hasta casa de Lord Darlington e intenta convencerla para que vuelva con su marido.

 Tokens:
El El DET DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art det
abanico abanicar NOUN NOUN__Gender=Masc|Number=Sing nsubj
de de ADP ADP__AdpType=Prep case
Lady Lady 

. . PUNCT PUNCT__PunctType=Peri punct

 
 SPACE _SP 
No No ADV ADV__Polarity=Neg advmod
creo creer VERB VERB__Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin ROOT
que que SCONJ SCONJ___ mark
lo el PRON PRON__Case=Acc|Gender=Masc|Number=Sing|Person=3|PronType=Prs obj
mande mandar VERB VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin ccomp
a a ADP ADP__AdpType=Prep case
ningún ninguno DET DET__Gender=Masc|Number=Sing|PronType=Ind det
sitio sitiar NOUN NOUN__Gender=Masc|Number=Sing obl
respondió responder VERB VERB__Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin ccomp
el el DET DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art det
artista artista NOUN NOUN__Number=Sing obj
, , PUNCT PUNCT__PunctType=Comm punct
echando echar VERB VERB__VerbForm=Ger advcl
la lo DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det
cabeza cabeza NOUN NOUN__Gender=Fem|Number=Sing obj
hacia hacia ADP ADP__AdpType=Prep case
atrás atrás ADV ADV___ advmod
de de ADP ADP__AdpTyp

, , PUNCT PUNCT__PunctType=Comm punct
una uno DET DET__Definite=Ind|Gender=Fem|Number=Sing|PronType=Art det
ración ración NOUN NOUN__Gender=Fem|Number=Sing nsubj
de de ADP ADP__AdpType=Prep case
estas este DET DET__Gender=Fem|Number=Plur|PronType=Dem det
empanadillas empanadilla NOUN NOUN__Gender=Fem|Number=Plur nmod
fritas frita ADJ ADJ__Gender=Fem|Number=Plur amod
suponen suponer VERB VERB__Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin ROOT
320 320 NUM NUM__NumForm=Digit nummod
calorías caloría NOUN NOUN__Gender=Fem|Number=Plur obj
y y CONJ CCONJ___ cc
20,2 20,2 NUM NUM__NumForm=Digit conj
g gramo CONJ CCONJ___ cc
de de ADP ADP__AdpType=Prep case
grasa graso NOUN NOUN__Gender=Fem|Number=Sing conj
. . PUNCT PUNCT__PunctType=Peri punct
Eso Eso PRON PRON__Number=Sing|PronType=Dem nsubj
sí sí ADV ADV___ advmod
, , PUNCT PUNCT__PunctType=Comm punct
con con ADP ADP__AdpType=Prep case
esto este PRON PRON__Number=Sing|PronType=Dem obl
no no ADV ADV__Polarity=Neg advmod
queremos quere

entrega entregar NOUN NOUN__Gender=Fem|Number=Sing ROOT
de de ADP ADP__AdpType=Prep case
The The PROPN PROPN___ nmod
Queen Queen PROPN PROPN___ flat
collection collection PROPN PROPN___ amod

 Entidades:
Queen ORG
El sexto MISC
News of the world MISC
We are the champions MISC
We will rock you MISC
Ambos MISC
Aunque parezca mentira MISC
Reino Unido LOC
Billboard de Estados Unidos ORG
Allí PER
Elektra Records ORG
A por lo bien MISC
Eso les proporcionó MISC
News of the world MISC
The Queen MISC

 Frases:
El número de especies ronda las cien, la mayoría originarias de Asia y un reducido número nativas de Europa, Norteamérica y África noroccidental.
Tanto especies como cultivares e híbridos se cultivan como ornamentales por la belleza y fragancia de su flor; pero también para la extracción de aceite esencial, utilizado en perfumería y cosmética, usos medicinales (fitoterapia) y gastronómicos.


Existe una enorme variedad de cultivares de rosa (más de 30 000) a partir de diversas hibridacion

Estudiamos el resultado en uno de los textos:

In [9]:
procesado = procesados[3]

print("\n Frases:")
for sent in procesado.sents:
    print(sent.text)


 Frases:
La Organización de las Naciones Unidas (ONU), o simplemente las Naciones Unidas (NN.
UU.
), es la mayor organización internacional existente.
Se creó para mantener la paz y seguridad internacionales, fomentar relaciones de amistad entre las naciones, lograr la cooperación internacional para solucionar problemas globales y servir de centro que armonice las acciones de las naciones.
Su sede está en Nueva York (Estados Unidos) y está sujeta a un régimen de extraterritorialidad.
También tiene oficinas en Ginebra (Suiza), Nairobi (Kenia) y Viena (Austria).


La ONU se rige por la Carta de las Naciones Unidas, que entró en vigor el 24 de octubre de 1945 y se firmó el 25 de junio del mismo año en la ciudad estadounidense de San Francisco, por 51 países, pocos meses antes del final de la Segunda Guerra Mundial.
En el preámbulo de la Carta se mencionan explícitamente las dos guerras mundiales.


In [10]:
print("\n Tokens:")
for token in procesado:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)


 Tokens:
La La DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det
Organización Organización PROPN PROPN___ ROOT
de de ADP ADP__AdpType=Prep case
las los DET DET__Gender=Fem|Number=Plur|PronType=Art det
Naciones Naciones PROPN PROPN___ flat
Unidas Unidas PROPN PROPN___ flat
( ( PUNCT PUNCT__PunctSide=Ini|PunctType=Brck punct
ONU ONU PROPN PROPN___ flat
) ) PUNCT PUNCT__PunctSide=Fin|PunctType=Brck punct
, , PUNCT PUNCT__PunctType=Comm punct
o o CONJ CCONJ___ cc
simplemente simplemente ADV ADV___ advmod
las los DET DET__Definite=Def|Gender=Fem|Number=Plur|PronType=Art det
Naciones Naciones PROPN PROPN___ conj
Unidas Unidas PROPN PROPN___ flat
( ( PUNCT PUNCT__PunctSide=Ini|PunctType=Brck punct
NN NN PROPN PROPN___ flat
. . PUNCT PUNCT__PunctType=Peri punct
UU UU PROPN PROPN___ ROOT
. . PUNCT PUNCT__PunctType=Peri punct
) ) PUNCT PUNCT__PunctSide=Fin|PunctType=Brck punct
, , PUNCT PUNCT__PunctType=Comm punct
es ser AUX AUX__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin

El paso a formas normales es mejor que el logrado con otros modelos si bien sigue generando algunos errores como por ejemplo asociar como la forma normal de la preposición para el verbo parir o el sustantivo sede con el verbo sedar.

In [11]:
print("\n Entidades:")
for ent in procesado.ents:
    print(ent.text, ent.label_)


 Entidades:
Organización de las Naciones Unidas ORG
ONU ORG
Naciones Unidas ORG
NN ORG
UU ORG
Nueva York LOC
Estados Unidos LOC
También LOC
Ginebra LOC
Suiza LOC
Nairobi LOC
Kenia LOC
Viena LOC
Austria LOC
ONU ORG
Carta de las Naciones Unidas ORG
San Francisco LOC
Segunda Guerra Mundial MISC
En el preámbulo de la Carta MISC


El único error que comete es caracterizar _También_ como una localización.

# LDA

Tras este breve análisis "exploratorio" se procede a un análisis LDA. Para ello es necesario eliminar las palabras vacías y pasarlas todas a minúsculas. Para ello cargamos en primer lugar las palabras vacías en español:

In [12]:
es_stop = get_stop_words('es')

Se recorre cada documentos pasando las palabras a minúsculas, obteniendo las formas normales y añadiéndolas a la lista si no son palabras vacías. Además se añade como restricción que tenga longitud mayor que uno porque lo más probable es que sea una palabra vacía o un signo de puntuación:

In [13]:
textos = []
palabras = []

for doc in doc_list:
    doc_min = doc.lower()
    doc = nlp(doc_min)
    for token in doc:
        if (not token.lemma_ in es_stop) and (len(token.lemma_)>1):
            palabras.append(token.lemma_)
    textos.append(palabras)
    palabras = []

En textos se almacenan los documentos ya procesados. Tras ello se procede a la construcción del diccionario de términos:

In [14]:
dictionary = corpora.Dictionary(textos)

Tras esto se genera la matriz de términos documentos:

In [15]:
corpus = [dictionary.doc2bow(texto) for texto in textos]

Finalmente se construye el modelo LDA. Fijamos cuatro clusters porque son en teoría los presentes en los documentos: alimentación, literatura, deporte y miscelánea:

In [16]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=4, id2word = dictionary, passes=50)

Visualizamos los cinco términos más frecuentes en cada tópico:

In [17]:
terminos_topic = ldamodel.print_topics(num_topics=4, num_words=5)

In [18]:
for terminos in terminos_topic:
    print(terminos)

(0, '0.021*"the" + 0.011*"do" + 0.011*"we" + 0.011*"news" + 0.011*"listo"')
(1, '0.022*"ser" + 0.020*"rosa" + 0.011*"windermere" + 0.011*"lady" + 0.011*"aceitar"')
(2, '0.028*"ser" + 0.017*"obrar" + 0.009*"vez" + 0.009*"hacer" + 0.009*"alguno"')
(3, '0.018*"ser" + 0.015*"unir" + 0.015*"nación" + 0.012*"haber" + 0.010*"parir"')


Para visualizar de manera correcta el modelo se emplea el módulo pyLDAvis:

In [19]:
pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)



/Users/arturosanchezpalacio/Entornos virtuales/notebooks/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [20]:
pyLDAvis.display(data)

Los clusters se encuentran bastante separados por la diversidad de los temas.  En el más pequeño se encuentran los textos relacionados con comida. En el de mayor tamaño los temas de cultura. Los otros dos entremezclan temas.